In [2]:
from Adafruit_IO import Client, Feed
from i2c_sensor_classes import Ky015Sensor
from aio_helper_classes import HabitatMonitorClient
import time

In [3]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

# Testing Reading from the Sensors 
no cloud here, trying to decouple cloud code and sensor reading code

In [4]:
KY015_ADDR = 8
SCL_PIN = 2
SDA_PIN = 3

liba = MicroblazeLibrary(base.PMODA, ['i2c'])
#custom class see 'src/i2c_sensor_classes.py'
ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

response = input('\t-Read Temp(y/n): ')
while response != 'n':
    temp, hum = ky015.get_temp_and_hum_data()
    print('\t\tTemp: {}C, Hum: {}%'.format(temp,hum))
    response = input('\t-Read Temp(y/n): ')

	-Read Temp(y/n): y
		Temp: 20.0C, Hum: 44.0%
	-Read Temp(y/n): y
		Temp: 20.02C, Hum: 44.0%
	-Read Temp(y/n): n


# Testing the Habitat Monitor class
this class extends the default Adafruit_IO Client, with functionality specific to out Habitat Monitor. If check the client_id and either sets the feeds to the correct Feed Objects, or if the client_id is new then new data feeds are created and associated with the passed client-id. 

In [5]:
UNAME = input('Enter username: ')
KEY = input('Enter Key: ')
client_id = 'joeys-pynq-board'

#custom class see "src/aio_helper_classes.py"
aio = HabitatMonitorClient(UNAME, KEY, client_id)    

Enter username: vsaah
Enter Key: aio_XdZP77Piq5DMQH8uhaKLF6SJfMOz
creating new HabitatMonitor...
HabitatMonitorClientsetup


In [11]:
response = 'y'
while response != 'n':
    response = input('\t-Read Data?(y/n): ')
    if response == 'n': break
    temp, hum = ky015.get_temp_and_hum_data()
    print('\t\tTemp: {}C, Hum: {}%'.format(temp,hum))
    response = input('\t-Publish to Cloud?(y/n): ')
    if response == 'n': break
    aio.send(aio.temperature_feed.key, temp)
    aio.send(aio.humidity_feed.key, hum)
    print('\t\tData Sent to Adafrui.io')

	-Read Data?(y/n): Y
		Temp: 20.02C, Hum: 44.0%
	-Publish to Cloud?(y/n): Y
		Data Sent to Adafrui.io
	-Read Data?(y/n): N
		Temp: 20.02C, Hum: 44.0%
	-Publish to Cloud?(y/n): n
